In [ ]:
import pandas as pd
from lmkgroup_ds_utils.db.connector import DB
from cheffelo_personalization.basket_preselector.data.customers import CustomerData
from cheffelo_personalization.basket_preselector.pipeline import (
    get_run_configs,
    get_company_configs,
)
from cheffelo_personalization.basket_preselector.run import load_data_from_adb
from cheffelo_personalization.basket_preselector.data.retriever import get_data_from_db
from cheffelo_personalization.basket_preselector.data.processor import process_raw_data
from cheffelo_personalization.basket_preselector.rules.engine import run_filter_rules

In [ ]:
year = 2023
week = 40
company_id = "6a2d0b60-84d6-4830-9945-58d518d27ac2"
num_quarantine_weeks = 4
local = True
config = "default"
company = "GL"
agreement_id = 1223454

In [ ]:
run_config = get_run_configs()
run_config

### Get data

In [ ]:
(
    df_customers,
    df_menu,
    df_recommendations,
    df_preference_rules,
    df_quarantined_dishes,
) = load_data_from_adb(
    local=local,
    company=company,
    year=year,
    week=week,
    config=config,
    run_config=run_config,
)

In [ ]:
df_customers, df_flex_products, df_recommendations = process_raw_data(
    df_customers, df_menu, df_recommendations, run_config
)

In [ ]:
customer = df_customers[df_customers["agreement_id"] == agreement_id].iloc[0]

In [ ]:
customer

In [ ]:
df_rec = None
if run_config["ranking"]["rec_engine"]:
    df_rec = df_recommendations.loc[
        df_recommendations["agreement_id"] == customer["agreement_id"]
    ]

In [ ]:
df_quarantined_dishes_for_customer = None
if run_config["ranking"]["quarantine"]:
    df_quarantined_dishes_for_customer = df_quarantined_dishes.loc[
        df_quarantined_dishes["agreement_id"] == customer["agreement_id"]
    ]

#### Run filter rules

In [ ]:
debug_summary: dict[str, int] = {}
possible_dishes, debug_summary = run_filter_rules(
    possible_dishes=df_flex_products,
    customer=customer,
    run_config=run_config,
    debug_summary=debug_summary,
)

## Dishes ranking

In [ ]:
possible_dishes["score"] = 0
possible_dishes

In [ ]:
df_rec

#### Rank dishes based on rec engine

In [ ]:
"""Rank dishes based on recommendation engine

Args:
    possible_dishes (pd.DataFrame): All possible dishes in menu
    df_rec (pd.DataFrame): dataframe of recommendation scores
    debug_summary (Dict): Summary of log

Returns:
    Tuple[pd.DataFrame, Dict]: possible dishes sorted by recommendation score and log summary
"""
if not df_rec.empty:
    debug_summary["ranking_rec_engine"] = "Has rec engine score"
    possible_dishes_with_rec_engine = possible_dishes.merge(
        df_rec[["product_id", "order_of_relevance", "order_of_relevance_cluster"]],
        on="product_id",
        how="left",
    )
    possible_dishes_with_rec_engine["score"] = possible_dishes_with_rec_engine["score"] + possible_dishes_with_rec_engine["order_of_relevance_cluster"]
    debug_summary[
        "ranking_rec_engine"
    ] = "Possible dishes reordered based on rec engine score"
    
    possible_dishes = possible_dishes_with_rec_engine.sort_values(by="score")

In [ ]:
possible_dishes.head()

### Rank dishes based on quarantine

In [ ]:
if not df_quarantined_dishes_for_customer.empty:
    possible_dishes["score"] = 
possible_dishes

In [ ]:
# Compare basket history vs dishes in menu, dishes that the customer has recently ordered or will recieve in weeks before, rank to lower position

# TODO: Implement with talks to recipes linking
# One change from PIM, dishes can be linked together, same dish but small differences such in protein (recipes_linking).

# Move dishes that has been bought last 3 weeks down 5 spots

In [ ]:
possible_dishes.loc[possible_dishes["main_recipe_id"].isin(df_quarantined_dishes_for_customer["main_recipe_id"]), "score"] += 5
possible_dishes